Okay! Let's get to the good stuff! Let's start the same way as we explored last time in explore_code_for_smiles:

In [1]:
from Drug import *
from parse_xml import *
from parse_xml import *

loc = "C:\\Users\\somat\\Downloads\\full database.xml"
_ = open(loc,encoding = "utf8")
a = _.read()
a = a.split("\n")

a = separate_by_drug(a)

drugs_library = []

drugs_list = [create_drug_object(x) for x in a]
small_molecules = list(filter(lambda x: x.lookup_attr("SMILES") != "No Data", drugs_list))
magic_n = 20
print(len(small_molecules))

9296


I remember making some functions to clean up the SMILES codes, so let's go and clean up our drugs.

In [2]:
from smiles_functions import *
unique_characters = create_unique_chars_histogram(small_molecules)
cleaned_molecules = clean_molecules_using_histogram(small_molecules,unique_characters, n=magic_n)


In [3]:
print(len(cleaned_molecules))

6995


I'm trying to do an analysis of the molecule's structure vs its function. The function is given by the ATC code, while the molecule's structure is given by its SMILES. 

Let's start with the ATC codes.

In [4]:
cleaned_molecules = list(filter(lambda x: x.atc_code[0] != "X", cleaned_molecules))
print(len(cleaned_molecules))

1587


What...? So few molecules left? Uh-oh. Looks like a lot of molecules didn't have atc codes listed... A serious (supervised) analysis is probably impossible with this little data, so I suppose I'll perform a toy example just to see if it's possible.

Anyway, lets see our class distribution out of what's left.

In [ ]:
labels = []
for mol in cleaned_molecules:
    should_add = True
    for i, label in enumerate(labels):
        if label[0] == mol.atc_code[0]:
            labels[i][1] += 1
            should_add = False
            break
    if should_add:
        labels.append([mol.atc_code[0], 0])
print(labels)

[['B', 42], ['L', 135], ['R', 129], ['H', 24], ['J', 183], ['S', 67], ['A', 150], ['V', 38], ['N', 307], ['M', 77], ['C', 218], ['D', 84], ['G', 78], ['P', 41]]


"N" encompasses all nervous system agents. That seems like a good target.
I'll do binary classification: nervous system agents vs non-nervous system agents. 

Firstly, I should make a function to encode SMILES. 

In [ ]:
def encode_SMILES_as_onehot(SMILES, character_list):
    encoded = []

    forged_smiles = bracket_forge(element_detector(SMILES))
    for char in forged_smiles:
        blank = [0 for x in range(magic_n)]
        for i, contained_character in enumerate(character_list):
            if contained_character[0] == char:
                blank[i] = 1
                break
        encoded.append(blank)
    return encoded
    

The next step is to convert every data point into features and labels. I'm also going to split the features/labels here into Nervous and Other because there will be a class imbalance. (more non-nervous than nervous). Also, let's set aside some test data.

In [ ]:
features_and_labels_N = []
features_and_labels_Other = []
for mol in cleaned_molecules:
    feature = encode_SMILES_as_onehot(mol.lookup_attr("SMILES"), unique_characters)
    label = mol.atc_code[0] == "N"
    label = [label, not label]
    if label[0]:
        features_and_labels_N.append([feature, label])
    else:
        features_and_labels_Other.append([feature, label])

In [ ]:
import random
random.shuffle(features_and_labels_N)
random.shuffle(features_and_labels_Other)
stop_N = int(len(features_and_labels_N)*0.1)
stop_O = int(len(features_and_labels_Other)*0.1)

features_and_labels_test = features_and_labels_N[:stop_N] + features_and_labels_Other[:stop_O]
features_and_labels_N = features_and_labels_N[stop_N:]
features_and_labels_Other = features_and_labels_Other[stop_O:]

print(len(features_and_labels_test))
print(len(features_and_labels_N))
print(len(features_and_labels_Other))


157
278
1152


Well, that's all we've got. Let's make it count! and build the RNN.

Actually, first, we have to pad each one-hot to the longest length in the batch. Let's make a function to do that.

In [ ]:
def pad_onehots(onehot_batch):
    o = onehot_batch
    onehot_lengths = [len(x) for x in onehot_batch]
    maximum_length = max(onehot_lengths)
    for i, _ in enumerate(o):
        while len(o[i]) < maximum_length:
            o[i].append([0 for x in range(magic_n)])
    return o, onehot_lengths
            

In [ ]:
import tensorflow as tf

sess= tf.Session()
features = tf.placeholder(tf.float32, [None, None, magic_n])
labels = tf.placeholder(tf.float32, [None, 2])
seq_lengths = tf.placeholder(tf.int32, [None])

layers_f = [tf.contrib.rnn.LSTMCell(unit) for unit in [1024,512]]
stacked_f = tf.contrib.rnn.MultiRNNCell(layers_f)

LSTM_output,_ = tf.nn.dynamic_rnn(stacked_f,features,dtype = tf.float32,sequence_length = seq_lengths)
indices = tf.stack([tf.range(tf.shape(LSTM_output)[0]),seq_lengths-1], axis=1)
last_output = tf.gather_nd(LSTM_output, indices)

hidden_layer = tf.layers.dense(inputs = last_output, units = 256, activation = tf.nn.relu)
final_layer = tf.layers.dense(inputs = hidden_layer, units = 2, activation = tf.nn.softmax)

prediction = tf.argmax(final_layer, axis=1)
correct_preds = tf.math.equal(tf.cast(prediction,tf.int32), tf.cast(tf.argmax(labels, axis=1), tf.int32))
accuracy = tf.reduce_sum(tf.cast(correct_preds,tf.float32)) / tf.cast(tf.shape(prediction)[0], tf.float32)

loss = tf.losses.softmax_cross_entropy(labels, final_layer)
opt = tf.train.AdamOptimizer().minimize(loss)
sess.run(tf.global_variables_initializer())

All this work... but will it blend?
I mean train.

In [ ]:
for z in range(100000):
    selected_samples = random.sample(features_and_labels_N, 20) + random.sample(features_and_labels_Other, 20)
    train_features, train_seq_lengths = pad_onehots([x[0] for x in selected_samples])
    train_labels = [x[1] for x in selected_samples]
    acc, _ = sess.run([accuracy, opt], feed_dict = {features:train_features, labels:train_labels, seq_lengths:train_seq_lengths})
    if z % 100 == 0:
 
        print("Train acc "+ str(acc))
    

Train acc 0.45
Train acc 0.5
